In [1]:
import pandas as pd

In [2]:
desc_df = pd.read_excel("../data/desc.xlsx")

In [3]:
risk_desc_df = desc_df[['project_id', 'project_address']]

In [4]:
#-*- coding: utf-8 -*-
import os
from appscript import *
import warnings
import webbrowser

warnings.filterwarnings('ignore')

# TEST CODE

osascript -e 'tell application "Google Chrome" to tell tab 1 of window 1
                open location "https://www.kickstarter.com/projects/1552697291/ufactory-3d-printing-shop/"
                delay 4
                set html_returned to missing value
                set html_returned to (execute javascript "document.documentElement.innerHTML")
                set myFile to open for access "/Users/jonghkim/dev_jhk/Research/Crowdfunding/Speech_Emotion/risk-perception/notebook/html/temp.html" with write permission
                write html_returned to myFile
                close access myFile
              end tell'   

osascript -e 'tell application "Google Chrome" to tell tab 1 of window 1
        if not (exists window 1) then reopen
        set URL to "https://www.kickstarter.com/projects/1552697291/ufactory-3d-printing-shop/"
        delay 8
        set html_returned to missing value
        set html_returned to (execute JavaScript "document.documentElement.innerHTML") 
        set myFile to open for access "/Users/jonghkim/dev_jhk/Research/Crowdfunding/Speech_Emotion/risk-perception/notebook/html/temp.html" with write permission
        write html_returned to myFile
        close access myFile        
        end tell'

In [5]:
def trick_save(url, fname):
        cmd = """osascript -e 'tell document 1 of application "Safari"
            set URL to "{url}"
            set the_state to missing value
            repeat until the_state is "complete"
                set the_state to (do JavaScript "document.readyState")
                delay 0.2
            end repeat
            delay 4
            set html_returned to missing value
            set html_returned to (do JavaScript "document.body.innerHTML")
            set myFile to open for access "{write_path}" with write permission
            write html_returned to myFile
            close access myFile
            end tell'""".format(url=url, write_path = fname)
        os.system(cmd)

In [6]:
from bs4 import BeautifulSoup

def parse_risk_text(fname):
    try:
        text = open(fname, "r").read()
        text = text.partition("Risks and challenges")[2]
        text = text.partition("Questions about this project?")[0]

        soup = BeautifulSoup(text, 'html.parser')
        matched = soup.findAll(attrs={'class':'js-risks-text'})[0].contents[0]

        return matched
    except:
        return ""

In [7]:
cwd = os.getcwd()

In [8]:
browser = webbrowser.get('safari')
browser.open("https://www.example.com")

True

In [9]:
safari = app("Safari")

In [12]:
import os
import tqdm
import time

risk_desc_list = []

for i, r in tqdm.tqdm(risk_desc_df.iterrows(), total=risk_desc_df.shape[0]):
    fpath = cwd + "/html/{}.html".format(i)
    try:
        if os.path.exists(fpath)==True:
            matched_text = parse_risk_text(fpath)

            if len(matched_text) ==0:
                url = 'https://www.kickstarter.com'+r['project_address']
                trick_save(url, fpath)
                matched_text = parse_risk_text(fpath)
                #time.sleep(3)            

        else:
            url = 'https://www.kickstarter.com'+r['project_address']
            trick_save(url, fpath)
            matched_text = parse_risk_text(fpath)
            #time.sleep(3)            

        risk_desc_list.append(matched_text)
    except:
        print("Exception at {}".format(i))
        risk_desc_list.append('')

 78%|███████▊  | 5913/7571 [02:43<00:08, 188.80it/s]

Exception at 5868


 79%|███████▉  | 5990/7571 [02:48<00:43, 36.09it/s]

Exception at 6010


100%|██████████| 7571/7571 [03:32<00:00, 35.68it/s]


In [13]:
risk_desc_df['text'] = risk_desc_list

In [14]:
risk_desc_df.to_pickle('risk_desc_df_trick.pkl')

In [15]:
risk_desc_df.to_csv('risk_desc_df_trick.csv', encoding='utf-8')

In [17]:
risk_desc_df.to_excel('risk_desc_df.xlsx', encoding='utf-8')